### XGBoost cloud training template for Diabetes Prediction

In [1]:
import numpy as np
import pandas as pd

import boto3
import re
import sagemaker
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Import s3 bucket name from .env file

import os
env_vars = !cat ./.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key]=value

In [3]:
bucket_name = os.environ['BUCKET_NAME']

In [ ]:
bucket_name

In [5]:
# bucket name and folders for Diabetes files in s3

training_file_key = 'diabetes/diabetes_train.csv'
validation_file_key = 'diabetes/diabetes_validation.csv'

s3_model_output_location = r's3://{0}/iris/model'.format(bucket_name)
s3_training_file_location = r's3://{0}/{1}'.format(bucket_name, training_file_key)
s3_validation_file_location = r's3://{0}/{1}'.format(bucket_name, validation_file_key)

In [ ]:
print(s3_model_output_location)
print(s3_training_file_location)
print(s3_validation_file_location)

In [7]:
# function to write files into s3

def write_to_s3(filename, bucket, key):
    with open(filename,'rb') as f:
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [8]:
write_to_s3('diabetes_train_imputed.csv',bucket_name,training_file_key)
write_to_s3('diabetes_validation_imputed.csv',bucket_name, validation_file_key)

In [9]:
diabetes_training_column_list_file_key = 'diabetes/diabetes_training_column_list.txt'
s3_training_column_list_file_location = r's3://{0}/{1}'.format(bucket_name, diabetes_training_column_list_file_key)

In [10]:
write_to_s3('diabetes_training_column_list.txt',bucket_name, diabetes_training_column_list_file_key)

### Training XGBoost Algorithm in Docker Image

#### using the AWS maintained xgboost image for every region

In [13]:
# Use spot instance

use_spot_instance = True
max_run = 3600
max_wait = 7200

job_name = 'xgboost-diabetes-v1'

checkpoint_s3_uri = None

if use_spot_instance:
    checkpoint_s3_uri = f's3://{bucket_name}/diabetes/checkpoints/{job_name}'

In [14]:
sess = sagemaker.Session()

In [15]:
role = get_execution_role()

In [ ]:
print(role)

In [17]:
container = sagemaker.image_uris.retrieve("xgboost",sess.boto_region_name, version='1.2-2')
print(f'Using XGBoost container:{container}')

Using XGBoost container:683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-2


### Build Model